In [232]:
#imports
import os
import requests
import pandas as pd
import json
import hvplot.pandas
import numpy as np
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import panel as pn
from pathlib import Path
import holoviews as hv
from holoviews import opts
import PyPDF2
from PyPDF2 import PDFFileReader

from dotenv import load_dotenv
from matplotlib import rcParams

%matplotlib "inline"

ImportError: cannot import name 'PDFFileReader' from 'PyPDF2' (C:\Users\dhaam\anaconda3\envs\dev\envs\pyvizenv\lib\site-packages\PyPDF2\__init__.py)

In [233]:
#read in all team stats csv
allteamstatscsv = Path("All Team Stats with and without Lebron.csv")
allteamstats = pd.read_csv(allteamstatscsv, index_col="Year")

#read in cavs 1 csv
cavs1csv = Path("Cavs 1 with LeBron.csv")
cavs_1_w_lebron = pd.read_csv(cavs1csv, index_col="Year")
cavs2csv = Path("Cavs 1 without LeBron.csv")
cavs_1_wo_lebron = pd.read_csv(cavs2csv, index_col = "Year")

#read in cavs 2 cavs
cavs3csv = Path("Cavs 2 with LeBron.csv")
cavs_2_w_lebron = pd.read_csv(cavs3csv, index_col="Year")
cavs4csv = Path("Cavs 2 without LeBron.csv")
cavs_2_wo_lebron = pd.read_csv(cavs4csv, index_col = "Year")

#read in heat csv
heat1csv = Path("Heat with LeBron.csv")
heat_w_lebron = pd.read_csv(heat1csv, index_col="Year")
heat2csv = Path("Heat without LeBron.csv")
heat_wo_lebron = pd.read_csv(heat2csv, index_col = "Year")

#read in lakers csv
lakers1csv = Path("Lakers with LeBron.csv")
lakers_w_lebron = pd.read_csv(lakers1csv, index_col="Year")
lakers2csv = Path("Lakers without LeBron.csv")
lakers_wo_lebron = pd.read_csv(lakers2csv, index_col = "Year")

#import impact score charts from PDF
#pdf = open('impact_score.pdf')
#PDF_read = PDFFileReader(pdf)
#first_page = PDF_read.getPage(0)
#print(first_page.extractText())


In [215]:
#sample dataframe
allteamstats.head()

,Mean PTS,Sum PTS,WR,Win Counts,Team Name
Year,,,,,
1999,90.97,7187,43.04,34,Cleveland Caveliers 1
2000,94.44,7650,44.44,36,Cleveland Caveliers 1
2001,92.27,7751,30.95,26,Cleveland Caveliers 1
2002,94.58,8039,27.06,23,Cleveland Caveliers 1
2003,92.10,4513,22.45,11,Cleveland Caveliers 1


In [235]:
#plot total wins v total points for games
wins_v_points = allteamstats.hvplot.scatter(x = 'Sum PTS',
                                           y = 'Win Counts',
                                           xlabel = 'Total Points Scored',
                                           ylabel = 'Total Games Won',
                                          width = 1000,
                                          height = 500,
                                           title = "Total Wins v Total Points Scored"
                                          )

#Plot team dataframes for comparison
cavs1_plot = pd.concat([cavs_1_wo_lebron, cavs_1_w_lebron])
cavs1_plots = cavs1_plot.hvplot.bar(x = "Year", 
                                    y = 'Mean PTS', 
                                    ylabel = 'Average Points', 
                                    title = 'Average Points Scored by Cavaliers 1999 - 2010 with LeBron v without LeBron', 
                                    color = 'Team Name', 
                                    cmap = ['maroon', 'navy'],
                                    width = 1000
                                   ).opts(axiswise=True)

cavs2_plot = pd.concat([cavs_2_wo_lebron, cavs_2_w_lebron])
cavs2_plots = cavs2_plot.hvplot.bar(x = "Year", 
                                    y = "Mean PTS", 
                                    ylabel = 'Average Points', 
                                    title = 'Average Points Scored by Cavaliers with LeBron v without LeBron', 
                                    color = 'Team Name', 
                                    cmap = ['black', 'gold'],
                                   width = 1000).opts(axiswise=True)

heat_plot = pd.concat([heat_wo_lebron, heat_w_lebron])
heat_plots = heat_plot.hvplot.bar(x = "Year", 
                                  y = "Mean PTS", 
                                  ylabel = 'Average Points', 
                                  title = 'Average Points Scored by Heat with LeBron v without LeBron', 
                                  color = 'Team Name', 
                                  cmap = ['black', 'red'],
                                 width = 1000).opts(axiswise=True)

lakers_plot = pd.concat([lakers_wo_lebron, lakers_w_lebron])
lakers_plots = lakers_plot.hvplot.bar(x = "Year", 
                                      y = "Mean PTS", 
                                      ylabel = 'Average Points', 
                                      title = 'Average Points Scored by Lakers with LeBron v without LeBron', 
                                      color = 'Team Name', 
                                      cmap = ['purple', 'gold'],
                                     width = 1000).opts(axiswise=True)

bars_as_columns = pn.Column(
        pn.pane.Markdown('##Average Team points scored with LeBron v without LeBron',style={'color':'white', 'font-family': "serif"}),
        cavs1_plots,
        cavs2_plots,
        heat_plots,
        lakers_plots
        )

#Create winrate dropdown plot
winrate_plot = allteamstats.hvplot.line(x='Year',
                                        y='WR',
                                        title = "Team's Yearly Winrate%", 
                                        xlabel="Year",
                                        ylabel="Win Rate",
                                        groupby = "Team Name", 
                                        rot=90,
                                        height=400, 
                                        width=500,
                                        color='c')
winrate_plot = winrate_plot * hv.VLine(2003)
#create plot to show total points scored.
sum_plot = allteamstats.hvplot.bar(x = 'Year', 
                                   y = 'Sum PTS',
                                   title="Team's Yearly Total Points Scored", 
                                   xlabel="Year",
                                   ylabel="Sum PTS",
                                   groupby = "Team Name", 
                                   rot=90,
                                   height=400, 
                                   width=500,
                                   color='red')


#combine
dashboard = pn.Tabs(
    ("Total Wins v Total Points Scored", wins_v_points),
    ("Total Points Scored with v without LeBron", sum_plot),
    ("Average Points Scored by Teams with v without LeBron",bars_as_columns),
    ("Team Win Rate with v without LeBron",winrate_plot)
   )

dashboard

Tabs
    [0] HoloViews(Scatter, name='Total Wins v T...)
    [1] Row(name='Total Points S...)
        [0] HoloViews(DynamicMap, name='Total Points S...)
        [1] Column
            [0] WidgetBox
                [0] Select(margin=(20, 20, 20, 20), name='Team Name', options=['Cleveland Caveliers 1', ...], value='Cleveland Caveliers 1', width=250)
            [1] VSpacer()
    [2] Column
        [0] Markdown(str, style={'color': 'white', ...})
        [1] HoloViews(Bars)
        [2] HoloViews(Bars)
        [3] HoloViews(Bars)
        [4] HoloViews(Bars)
    [3] Row(name='Team Win Rate w...)
        [0] HoloViews(DynamicMap, name='Team Win Rate w...)
        [1] Column
            [0] WidgetBox
                [0] Select(margin=(20, 20, 20, 20), name='Team Name', options=['Cleveland Caveliers 1', ...], value='Cleveland Caveliers 1', width=250)
            [1] VSpacer()

In [ ]:
#serve up the dashboard
dashboard.servable()